# Y~G ACGT Probabilities 1d
> 

In [58]:
import numpy as np
import pandas as pd

from EnvDL.core import ensure_dir_path_exists 
from EnvDL.dlfn import g2fc_datawrapper, BigDataset, plDNN_general
from EnvDL.dlfn import ResNet1d, BasicBlock1d
from EnvDL.dlfn import LSUV_

import torch
import torch.nn.functional as F # F.mse_loss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger

In [59]:
# Run settings: 
max_epoch = 20
batch_size = 24

In [60]:
use_gpu_num = 0

device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


In [63]:
cache_path = '../nbs_artifacts/02.22_g2fc_G_ACGT_conv1d/'
ensure_dir_path_exists(dir_path = cache_path)

## Load data

In [64]:
X = g2fc_datawrapper()
X.set_split()
X.load_all(name_list = ['obs_geno_lookup', 'YMat', 'ACGT',], store=True) 

X.calc_cs('YMat', version = 'np', filter = 'val:train')

Loading and storing default `phno`.


In [65]:
training_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs  = X.get('val:train',       ops_string='   asarray from_numpy      '),
    lookup_geno = X.get('obs_geno_lookup', ops_string='   asarray from_numpy      '),
    y =           X.get('YMat',            ops_string='cs asarray from_numpy float')[:, None],
    G =           X.get('ACGT',            ops_string='           from_numpy float'),
    G_type = 'raw',
    send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = True
)


validation_dataloader = DataLoader(BigDataset(
    lookups_are_filtered = False,
    lookup_obs =  X.get('val:test',        ops_string='   asarray from_numpy      '),
    lookup_geno = X.get('obs_geno_lookup', ops_string='   asarray from_numpy      '),
    y =           X.get('YMat',            ops_string='cs asarray from_numpy float')[:, None],
    G =           X.get('ACGT',            ops_string='           from_numpy float'),
    G_type = 'raw',
    send_batch_to_gpu = 'cuda:0'
    ),
    batch_size = batch_size,
    shuffle = False
)

## Test Models

In [81]:
def Conv1D_Max_block(in_channels, out_channels, kernel_size, stride):
    block = nn.Sequential(
        nn.Conv1d(
            in_channels= in_channels, # second channel
            out_channels= out_channels,
            kernel_size= kernel_size,
            stride= stride
        ), 
        nn.MaxPool1d((kernel_size,), stride=stride)
    )
    return(block)


In [83]:
from EnvDL.dlfn import Linear_block

In [93]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()    

#         def Linear_block(in_size, out_size, drop_pr):
#             block = nn.Sequential(
#                 nn.Linear(in_size, out_size),
#                 nn.ReLU(),
#                 nn.Dropout(drop_pr)
#             )
#             return(block)         
        
        
#         def Conv1D_Max_block(in_channels, out_channels, kernel_size, stride):
#             block = nn.Sequential(
#                 nn.Conv1d(
#                     in_channels= in_channels, # second channel
#                     out_channels= out_channels,
#                     kernel_size= kernel_size,
#                     stride= stride
#                 ), 
#                 nn.MaxPool1d((kernel_size,), stride=stride)
#             )
#             return(block)
        
        self.x_network = nn.Sequential(
            Conv1D_Max_block(4, 4, 3, 2),
            Conv1D_Max_block(4, 4, 3, 2),
            Conv1D_Max_block(4, 4, 3, 2),
            Conv1D_Max_block(4, 4, 3, 2),
            Conv1D_Max_block(4, 4, 3, 2),
            Conv1D_Max_block(4, 4, 3, 2)
        )
        
    
        self.x_pred = nn.Sequential(
                nn.Flatten(),           
                Linear_block(in_size = (4*29), out_size = 32, drop_pr = 0.3),
                nn.Linear(32, 1)
            )

    def forward(self, x):
        out = self.x_network(x)
        pred = self.x_pred(out)
        return pred#, out

#     def forward(self, x):
#         x_out = self.x_network(x)
#         return x_out

model = NeuralNetwork().to(device)

# model(next(iter(training_dataloader))[1][0:5]).shape

In [95]:
LSUV_(model, next(iter(training_dataloader))[1])

Applying orthogonal init (zero init if dim < 2) to params in 8 module(s).
Applying LSUV to 8 module(s) (up to 10 iters per module):
Module  0 after  2 itr(s) | Mean: -0.485 | Std: 1.000 | <class 'torch.nn.modules.conv.Conv1d'>
Module  1 after  2 itr(s) | Mean: -0.047 | Std: 1.000 | <class 'torch.nn.modules.conv.Conv1d'>
Module  2 after  2 itr(s) | Mean: -0.227 | Std: 1.000 | <class 'torch.nn.modules.conv.Conv1d'>
Module  3 after  2 itr(s) | Mean: -0.686 | Std: 1.000 | <class 'torch.nn.modules.conv.Conv1d'>
Module  4 after  1 itr(s) | Mean:  0.067 | Std: 0.997 | <class 'torch.nn.modules.conv.Conv1d'>
Module  5 after  2 itr(s) | Mean: -0.214 | Std: 1.000 | <class 'torch.nn.modules.conv.Conv1d'>
Module  6 after  1 itr(s) | Mean: -0.070 | Std: 0.955 | <class 'torch.nn.modules.linear.Linear'>
Module  7 after  6 itr(s) | Mean:  2.065 | Std: 1.092 | <class 'torch.nn.modules.linear.Linear'>


In [96]:
DNNG = plDNN_general(model)     
optimizer = DNNG.configure_optimizers()

logger = TensorBoardLogger("tb_logs", name="g-acgt-conv1")
trainer = pl.Trainer(max_epochs=max_epoch, logger=logger)

trainer.fit(model=DNNG, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/kickd/miniconda3/envs/fastai/lib/python3.11/si ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type          | Params
---------------------------------------
0 | mod  | Neur

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/kickd/miniconda3/envs/fastai/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
# torch.save(DNNG.mod, cache_path+'g-acgt-conv1'+'.pt')